In [ ]:
pip install pm4py

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
  Created wheel for stringdist: filename=StringDist-1.0.9-cp37-cp37m-linux_x86_64.whl size=23594 sha256=54f26c9bc33eb8fab8d3f2d90a4c8423db1eaa761db9c39f1371285f4959cd6f
  Stored in directory: /root/.cache/pip/wheels/d7/9c/d4/63bc3d8931de0980b9e4a724dea290bb40b8b1b2bd6227c8da
Successfully built stringdist


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import pareto

import pm4py

In [ ]:
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.objects.log.importer.xes import importer as xes_importer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# test gdrive
with open('/content/gdrive/My Drive/_ComputerScience/README.txt') as f:
  lines = f.readlines()
  print(lines)


['The tenth International Business Process Intelligence Challenge is again co-located with ICPM. This challenge provides participants with a real-life event log, and challenges them to analyze these data using whatever techniques available, focusing on one or more of the process owner’s questions or proving other unique insights into the process(es) captured in the event log.\n', '\n', 'In many organizations, staff members travel for work. They travel to customers, to conferences or to project meetings and these travels are sometimes expensive. As an employee of an organization, you do not have to pay for your own travel expenses, but the company takes care of them.\n', '\n', 'For this year’s Business Process Intelligence Challenge, we collected data from the reimbursement process at TU/e. The files contain data from 2017 (only two departments) and 2018 the full TU/e.\n', '\n', 'The data is split into travel permits and several request types, namely domestic declarations, international

In [ ]:
import gzip
filename = '/content/gdrive/My Drive/_ComputerScience/BusinessInformationSystems/BPI_challenge_2020/DomesticDeclarations/DomesticDeclarations.xes.gz'
log = xes_importer.apply(filename)

parsing log, completed traces ::   0%|          | 0/10500 [00:00<?, ?it/s]

In [ ]:
print(log[0]) #prints the first trace of the log
print(log[0][0])

{'attributes': {'id': 'declaration 86791', 'concept:name': 'declaration 86791', 'BudgetNumber': 'budget 86566', 'DeclarationNumber': 'declaration number 86792', 'Amount': 26.85120450862128}, 'events': [{'id': 'st_step 86794_0', 'org:resource': 'STAFF MEMBER', 'concept:name': 'Declaration SUBMITTED by EMPLOYEE', 'time:timestamp': datetime.datetime(2017, 1, 9, 9, 49, 50, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'org:role': 'EMPLOYEE'}, '..', {'id': 'dd_declaration 86791_20', 'org:resource': 'SYSTEM', 'concept:name': 'Payment Handled', 'time:timestamp': datetime.datetime(2017, 1, 12, 17, 31, 22, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'org:role': 'UNDEFINED'}]}
{'id': 'st_step 86794_0', 'org:resource': 'STAFF MEMBER', 'concept:name': 'Declaration SUBMITTED by EMPLOYEE', 'time:timestamp': datetime.datetime(2017, 1, 9, 9, 49, 50, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'org:role': 'EMPLOYEE'}
